# Express




### Basic Server




#### setup code

How to use nodejs express?

express service?

express website?


In [ ]:
const express = require('express')
const cors = require('cors')
const router = express.Router();
const cookieParser = require('cookie-parser');
const loadRoutes = importer.import('express router')

process.env['NODE_TLS_REJECT_UNAUTHORIZED'] = 0;

const TIMEOUT = 30 * 1000
let ROOT = process.env.DEFAULT_ROOT
let DEFAULT = process.env.DEFAULT_ROOT

let server

async function startServer(services) {
  if (!services) {
    services = []
  }
  if (typeof services == 'string') {
    services = [services]
  }
  let codeCells = services.map(service => importer.interpret(service)).flat(1)
  var app = express()
  app.use(cors({
    credentials: true,
    origin: '*'
  }))
  app.use(cookieParser());
  app.use(express.json({ limit: '20mb' }))
  app.use(express.urlencoded({ extended: true }));
  for (let i = 0; i < codeCells.length; i++) {
    let { root, defaultRoute } = loadRoutes(app, router, codeCells[i])
    if (!ROOT && root) {
      ROOT = root
    }
    if (!DEFAULT && defaultRoute) {
      DEFAULT = defaultRoute
    }
  }
  app.get('/import/:query', async function (req, res, next) {
    let service = req.params['query']
    let codeCells = importer.interpret(service)
    let thisRoute
    let thisDefault
    let thisRoot
    if (typeof codeCells.markdown != 'undefined') {
      codeCells = [codeCells]
    }
    for (let i = 0; i < codeCells.length; i++) {
      let { route, root, defaultRoute } = loadRoutes(app, router, codeCells[i])
      if (!thisRoute && route) {
        thisRoute = route
      }
      if (!thisRoot && root) {
        thisRoot = root
      }
      if (!thisDefault && defaultRoute) {
        thisDefault = defaultRoute
      }
    }
    let route = thisRoot || thisDefault || thisRoute || ROOT || DEFAULT
    if(route === true) {
      console.log('error route not detected:' + service)
      return next()
    }
    return res.redirect(route)
  })
  router.all('/', function (req, res, next) {
    if (ROOT) {
      return res.redirect(ROOT)
    } else {
      return next()
    }
  })
  router.all('*', function (req, res, next) {
    if (DEFAULT) {
      return res.redirect(DEFAULT)
    } else {
      return next()
    }
  })
  app.use(router)
  let port = process.env.EXPRESS_PORT || 8080
  server = app.listen(port, () => console.log('Listening on http://localhost:' + port))

  server.keepAliveTimeout = TIMEOUT;
  server.headersTimeout = TIMEOUT;
}

function closeExpress(activity) {
  if (activity) {
    return
  }
  if (server) {
    server.close()
    server = null
  }
}

module.exports = {
  startServer,
  closeExpress,
}



### express rpc service

express rpc service?


In [ ]:

async function serviceHandler(service, req, res, next) {
  const getParameters = await importer.import('function parameters')
  let codeCell = await importer.interpret(service)
  if (!codeCell) {
    return next()
  }
  if (codeCell.language == 'css') {
    res.setHeader('content-type', 'text/css');
    return res.send(codeCell.code)
  }
  if (codeCell.language == 'html') {
    return res.send(codeCell.code)
  }
  let serviceFunction
  try {
    serviceFunction = await importer.import(service)
  } catch (e) {
    return res.status(500).send(e)
  }
  if (typeof serviceFunction == 'object' && typeof serviceFunction[Object.keys(serviceFunction)[0]] == 'function') {
    serviceFunction = serviceFunction[Object.keys(serviceFunction)[0]]
  }
  // TODO: support multiple languages parameters more nicely than importer.run()
  let isRPC = true
  let parameters
  if (codeCell.language == 'javascript') {
    parameters = getParameters(codeCell.code).slice(1)
    if ((parameters.includes('res') || parameters.includes('response'))
      && (parameters.includes('req') || parameters.includes('request'))) {
      isRPC = false
    } else {
      isRPC = true
    }
  }
  // TODO python

  // TODO C#

  try {
    if (isRPC && parameters) {
      let inputs = []
      for (let i = 0; i < parameters.length; i++) {
        inputs.push(req.body[parameters[i]] || req.params[parameters[i]] || req.query[parameters[i]] || req.cookies[parameters[i]])
      }
      let result = await serviceFunction.apply(null, inputs)
      if (typeof result == 'object')
        return res.json(result)
      else
        return res.send(result)
    } else {
      return await serviceFunction(req, res, next)
    }
  } catch (e) {
    return res.status(500).send(e)
  }
}

module.exports = serviceHandler


### express auto router

express router?


In [ ]:
const serviceHandler = importer.import('express rpc service')
const express = require('express')

process.env['NODE_TLS_REJECT_UNAUTHORIZED'] = 0;

const SCAN_ENVIRONMENT = /([A-Z_\[\]]*)\s*[\:-=]+\s*(.*?)\s*(\n|$)/g
const LOADED = {}

function loadRoutes(app, router, codeCell) {
  let env = {}
  while ((match = SCAN_ENVIRONMENT.exec(codeCell.markdown.join(''))) !== null) {
    if (match[1].includes('[]')) {
      if (typeof env[match[1]] == 'undefined') {
        env[match[1]] = []
      }
      env[match[1]].push(match[2])
    } else {
      env[match[1]] = match[2]
    }
  }
  let routes = env['ROUTE']
  if (typeof routes == 'string') {
    routes = [env['ROUTE']]
  }
  if (routes) {
    for (let i = 0; i < routes.length; i++) {
      console.log('creating route:', routes[i], (env['STATIC'] ? env['STATIC'] : codeCell.questions[0]))
      if(LOADED[codeCell.id]) {
        continue
      }
      LOADED[codeCell.id] = true

      if (env['AUTHENTICATE']) {
        let authenticate = eval(env['AUTHENTICATE'])
        if (env['STATIC']) {
          if (typeof env['STATIC'] == 'string') {
            app.use(routes[i], authenticate, express.static(eval(env['STATIC'])))
          } else {
            app.use(routes[i], authenticate, express.static(eval(env['STATIC'][i])))
          }
        } else if (env['METHOD'] == 'all') {
          router.all(routes[i], authenticate, serviceHandler.bind(null, codeCell.questions[0]))
        } else if (env['METHOD'] == 'post') {
          app.post(routes[i], authenticate, serviceHandler.bind(null, codeCell.questions[0]))
        } else {
          app.get(routes[i], authenticate, serviceHandler.bind(null, codeCell.questions[0]))
        }
      } else {
        if (env['STATIC']) {
          if (typeof env['STATIC'] == 'string') {
            app.use(routes[i], express.static(eval(env['STATIC'])))
          } else {
            app.use(routes[i], express.static(eval(env['STATIC'][i])))
          }
        } else if (env['METHOD'] == 'all') {
          router.all(routes[i], serviceHandler.bind(null, codeCell.questions[0]))
        } else if (env['METHOD'] == 'post') {
          app.post(routes[i], serviceHandler.bind(null, codeCell.questions[0]))
        } else {
          app.get(routes[i], serviceHandler.bind(null, codeCell.questions[0]))
        }
      }
    }

    if (env['ROOT'] && env['ROOT'] == 'true') {
      return { root: routes[0] }
    }
    if (env['DEFAULT'] && env['DEFAULT'] == 'true') {
      return { defaultRoute: routes[0] }
    }
    return { route: routes[0] }
  }
  return {}
}

module.exports = loadRoutes



#### initialize

How to start a proxy using express?

How to start a proxy on {port}?



In [ ]:
if (typeof server != 'undefined') {
    server.close();
}
var app = express();
app.use('/', function (req, res) {
    var url = host + req.url;
    req.pipe(request(url)).pipe(res);
});

$$.async();
try {
    var server = app.listen(port, () => $$.done('server up and running on port ' + port));
    server.on('error', (e) => $$.done(e));
} catch (e) {
    $$.done(e);
}




#### send user a link to itself

How to display the output from express?

How to test express is working with HTML?

In [ ]:
$$.mime({
    'text/html': `
<iframe id="sosmethod" name="sosmethod" 
    style="height:600px; width:100%; border:none;" 
    src="http://${host}:${port}/"></iframe>`
});


#### dom query using JSDOM

How to run a DOM query on a remote HTML page?

In [ ]:
$$.async();
var images = [];
new Promise((resolve, reject) => {
    request('http://' + host + ':' + port, (err, res, body) => {
        if (err) {
            reject(err)
        }
        else {
            resolve(body)
        }
    });
}).then((body) => {
    var nodeList = (new JSDOM(body)).window.document.querySelectorAll(query);
    return Array.prototype.slice.call(nodeList, 0)
        .map(i => i.getAttribute('src'));
}).then(sources => {
    images = sources;
    $$.done(sources);
}).catch(e => $$.done(e));


#### extract images

How to save a list of image elements?

How to display a list of image elements in markdown?

How to {output} image elements in the requested {format}?


In [ ]:
var html = '';
var htmlPrint = '';
var requests = images.map((src) => {
    var filename = src.split('/').pop();
    return new Promise((resolve, reject) => {
        request(src, {encoding: 'binary'}, (err, res, body) => {
            if (err) {
                reject(err)
            }
            else {
                resolve(body)
            }
        });
    }).then(body => {
        return new Promise((resolve, reject) => {
            fs.writeFile(
                path.join(output, filename),
                body,
                'binary',
                function (err) {
                    if (err) reject(err);
                    else resolve({filename: filename, data: body});
                });
        })
    });
});
$$.async();
Promise.all(requests).then(images => {
    images.forEach((i) => {
        var filename = i.filename.split('/').pop();
        var ext = mime.lookup(i.filename);
        html += '<img src="data:' + ext + ';base64,' + (new Buffer(i.data, 'binary')).toString('base64') + '" />';
        htmlPrint += '<li><img src="/assets/' + filename + '" /></li>\n';
    });
    $$.mime({'text/markdown': 'Usage:\n\n```html\n' + htmlPrint + '\n```\nOutput:\n' + html});
}).catch(e => $$.done(e));


#### stop server

How to stop express server?

In [ ]:
if (typeof server !== 'undefined') {
    server.close();
}




## RPC Wrapper



#### the code

express rpc wrapper?

ROUTE[] = /rpc

ROUTE[] = /rpc/:function

METHOD = all


In [ ]:
var importer = require('../Core');
var { getResult } = importer.import(['get rpc permissions', 'rpc result']);
async function rpcHandler(req, res) {
    try {
        let r = getResult({
            command: req.body['function'] || req.query['function'] || req.params['function'],
            result: importer.interpret(req.body['function'] || req.query['function']),
            body: req.method === 'POST' ? req.body : req.query,
            circles: ['Public']
        })
        res.status(200).send(JSON.stringify(r, null, 4));
    } catch (e) {
        const resultError = Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key];
            return alt;
        }, {});
        console.log(e);
        res.status(500).send(JSON.stringify(resultError, null, 4))
    }
}

module.exports = rpcHandler



#### test mock

old BS I wrote for a job that wasn't appreciated.

zuora eloqua express mock?


In [ ]:
var bodyParser = require('body-parser');
var express = require('express');

var selenium = express();
var server = require('http').createServer(selenium);
selenium.use(bodyParser.json());    // to support JSON-encoded bodies
selenium.use(bodyParser.urlencoded({// to support URL-encoded bodies
    extended: true
}));

var router = express.Router();


// auth endpoints
router.post('/auth/oauth2/authorize', (req, res) => { 
    res.send('');
});

router.post('/auth/oauth2/token', (req, res) => {
    res.send({
        "access_token": "access_token",
        "token_type": "bearer",
        "expires_in": 28800,
        "refresh_token": "refresh_token",
    })
});


// eloqua endpoints
router.post('/bulk/2.0/contacts/imports', (req, res) => {
    res.send({
        uri: '/imports/1234'
    });
});
router.post('/bulk/2.0/imports/:importId/data', (req, res) => {
    res.status(204);
    res.send({
        uri: '/imports/1234'
    });
});
router.post('/bulk/2.0/syncs', (req, res) => {
    res.send({
        uri: '/sync/1234'
    });
});
var alternateImportStatus = true;
router.get('/bulk/2.0/sync/:syncId', (req, res) => {
    res.send({
        status: alternateImportStatus ? 'active' : 'success',
    });
    alternateImportStatus = !alternateImportStatus;
});


// eloqua custom data object
// as opposed to /bulk/2.0/customobjects/imports which just lists all the imports that have been performed on the CDO
router.post('/bulk/2.0/customobjects/:objectId/imports', (req, res) => {
    res.send({
        uri: '/imports/1234'
        // might be? /customobjects/:objectId/imports/:importId
    });
});




// zuora endpoints
router.post('/object/export', (req, res) => {
    res.send({
        Id: '1234'
    });
});
var alternateExportStatus = true;
router.get('/object/export/:exportId', (req, res) => {
    res.send({
        Status: alternateExportStatus ? 'Processing' : 'Completed',
        FileId: '1234'
    });
    alternateExportStatus = !alternateExportStatus;
});
router.get('/files/:fileId', (req, res) => {
    res.send(`RatePlanCharge.Description,Account.Id
support,123456
premium,654321
`);
});

selenium.use(router);
var app;
function restart() {
    if(typeof app !== 'undefined') {
        app.close();
    }
    app = server.listen(18888).on('error', e => {
        if(e.code !== 'EADDRINUSE') {
            throw e;
        }
    });
    return app;
}
restart();
app.restart = restart;
module.exports = app;

process.on ('SIGTERM', () => {app.close(); process.exit(0)});
process.on ('SIGINT', () => {app.close(); process.exit(0)});



## Tools



### Directory to HTML

directory to html?


In [ ]:
const fs = require('fs');
const path = require('path');

function directoryToHtml(dirPath, back, session, bodyOnly) {
  let files = fs.readdirSync(dirPath)

  files.sort()
  files.sort((a, b) => {
    let aIsDir = fs.statSync(path.join(dirPath, a)).isDirectory()
    let bIsDir = fs.statSync(path.join(dirPath, b)).isDirectory()

    if (aIsDir && !bIsDir) return -1; // a is dir, b is file
    if (!aIsDir && bIsDir) return 1; // b is dir, a is file

    return a.localeCompare(b);
  })
  let htmlOutput = `<html>
<head>
<style>
ul {
  list-style: none;
  padding: 0;
  margin: 0;
}

li {
  padding: 10px;
  border-bottom: 1px solid #ccc;
}

li:last-child {
  border-bottom: none;
}

i {
  font-size: 18px;
  margin-right: 10px;
  color: #666;
}

span {
  font-size: 16px;
  color: #333;
}

ul {
  padding-left: 20px;
}

ul li {
  padding-left: 10px;
}
</style>
</head>  
<body><h1>Directory Listing</h1><ul>`

  if(bodyOnly) 
    htmlOutput = `<ul>`;

  if(back) {
    htmlOutput += `<li><a href="../?t=${Date.now()}${session ? ('&session=' + session): ''}">Parent directory/</a></li>`;
  }

  files.forEach((file) => {
    const filePath = path.join(dirPath, file);
    const fileStat = fs.statSync(filePath);

    if(file[0] == '.' || file.includes('porn') || file.includes('nsfw') || file.includes('naked') || file.includes('x-rated')) {
      return
    }

    if (fileStat.isDirectory()) {
      htmlOutput += `<li><a href="${encodeURIComponent(file).replaceAll('(', '%28').replaceAll(')', '%29')}/?t=${Date.now()}${session ? ('&session=' + session): ''}">${file}/</a></li>`;
    } else {
      htmlOutput += `<li><a href="${encodeURIComponent(file).replaceAll('(', '%28').replaceAll(')', '%29')}?t=${Date.now()}${session ? ('&session=' + session): ''}">${file}</a></li>`;
    }
  });

  if(bodyOnly) 
    htmlOutput += '</ul>';
  else
    htmlOutput += '</ul></body></html>';

  return htmlOutput
}

module.exports = directoryToHtml



### Directory Listing

node express directory handler?

DEFAULT = true

AUTHENTICATE = importer.functionOrDefault(importer.import(process.env.DEFAULT_AUTHENTICATOR))



In [ ]:
const path = require('path')
const fs = require('fs')
const BASE_DIRECTORY = process.env.BASE_DIRECTORY || process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
const directoryToHtml = importer.import('directory to html')

async function handleDirectory(req, res, next) {
  let basePath = path.resolve(BASE_DIRECTORY)

  let combined = path.join(basePath, decodeURIComponent(req.path))
  console.log(combined, req.authenticated)

  if(!fs.existsSync(combined)) {
    return next()
  }

  if(path.extname(combined).toLocaleLowerCase() == '.mp3') {

    if(req.url.includes('?raw=true')) {
      return res.sendFile(combined)
    }

    return res.send(importer.interpret('html music player').code
      .replaceAll('{FILE_URL}', req.path)
      .replaceAll('{SESSION}', req.authenticated ? req.authenticated : ''))
  }

  if((req.authenticated || combined.includes('stable-diffusion-webui/outputs')) 
    && (path.extname(combined).toLocaleLowerCase() == '.png' 
    || path.extname(combined).toLocaleLowerCase() == '.jpg' 
    || path.extname(combined).toLocaleLowerCase() == '.jpeg')
  ) {

    if(req.url.includes('?raw=true')) {
      return res.sendFile(combined)
    }

    return res.send(importer.interpret('html image viewer').code
      .replaceAll('{FILE_URL}', req.path)
      .replaceAll('{SESSION}', req.authenticated ? req.authenticated : ''))
  }

  if(req.authenticated && fs.statSync(combined).isFile()) {
    return res.sendFile(combined)
  }

  if(!fs.statSync(combined).isDirectory()) {
    return res.status(403).send('Forbidden')
  }

  let html = await directoryToHtml(combined, req.path.split('/').length > 2, req.authenticated)

  return res.send(html)
}

module.exports = {
  handleDirectory,
  BASE_DIRECTORY
}



### Music Player

html music player?


In [ ]:
<head>
<style>
ul {
  list-style: none;
  padding: 0;
  margin: 0;
}

li {
  padding: 10px;
  border-bottom: 1px solid #ccc;
}

li:last-child {
  border-bottom: none;
}

i {
  font-size: 18px;
  margin-right: 10px;
  color: #666;
}

span {
  font-size: 16px;
  color: #333;
}

ul {
  padding-left: 20px;
}

ul li {
  padding-left: 10px;
}
</style>
<link rel="stylesheet" type="text/css" href="/amplitude.css" />
<script src="/amplitude.js"></script>
</head>  
<body><h1>Directory Listing</h1><ul>
<li><a href="./?session={SESSION}">Parent directory/</a></li>
</ul>
<div id="visualizations-player">
  <div class="top-container">
    <img class="now-playing-album-art" id="large-now-playing-album-art" data-amplitude-song-info="cover_art_url"/>
    <div class="amplitude-visualization" id="large-visualization">

    </div>
    <div class="visualization-toggle visualization-on"></div>
    <div class="amplitude-shuffle"></div>
    <div class="amplitude-repeat"></div>
  </div>

  <div class="meta-data-container">
    <span class="now-playing-name" data-amplitude-song-info="name"></span>
    <span class="now-playing-artist-album">
      <span class="now-playing-artist" data-amplitude-song-info="artist"></span> - <span class="now-playing-album" data-amplitude-song-info="album"></span>
    </span>
  </div>

  <div class="amplitude-wave-form">

  </div>
  <input type="range" class="amplitude-song-slider" id="global-large-song-slider"/>

  <div>
    <span class="amplitude-current-time"></span>

    <span class="amplitude-time-remaining"></span>
  </div>

  <div class="control-container">
    <div class="amplitude-prev">

    </div>

    <div class="amplitude-play-pause amplitude-paused">

    </div>

    <div class="amplitude-next">

    </div>
  </div>

  <div class="song-navigation">
    <input type="range" class="amplitude-song-slider"/>
  </div>

  <div class="arrow-up">
    <img src="https://521dimensions.com/img/open-source/amplitudejs/examples/visualizations/arrow-up.svg" class="arrow-up-icon"/>
  </div>

  <div id="visualizations-player-playlist">
    <div class="top-arrow">
      <img src="https://521dimensions.com/img/open-source/amplitudejs/examples/visualizations/arrow-down.svg" class="arrow-down-icon"/>
    </div>

    <div class="top">
      <span class="playlist-title">Songs</span>
      <div class="amplitude-repeat">

      </div>
      <div class="amplitude-shuffle">

      </div>
    </div>

    <div class="songs-container">
      <div class="song amplitude-song-container amplitude-play-pause"  data-amplitude-song-index="0">
        <span class="song-position">01</span>
        <img class="song-album-art" data-amplitude-song-info="cover_art_url" data-amplitude-song-index="0"/>
        <div class="song-meta-data-container">
          <span class="song-name" data-amplitude-song-info="name" data-amplitude-song-index="0"></span>
          <span class="song-artist" data-amplitude-song-info="artist" data-amplitude-song-index="0"></span>
        </div>
      </div>
    </div>

    <div class="active-audio">
      <img class="cover-art-small" data-amplitude-song-info="cover_art_url"/>

      <div class="active-audio-right">
        <span class="song-name" data-amplitude-song-info="name"></span>

        <div class="active-audio-controls">
          <div class="amplitude-prev"></div>
          <div class="amplitude-play-pause"></div>
          <div class="amplitude-next"></div>
        </div>
      </div>
    </div>
  </div>
</div>
<script>
  Amplitude.init({
		songs: [
			{
				"name": "Song Name 1",
				"artist": "Artist Name",
				"album": "Album Name",
				"url": "{FILE_URL}?raw=true" + ('{SESSION}'.length > 0 ? ('&session={SESSION}') : ''),
				"cover_art_url": "/cover/art/url.jpg"
			},
    ]
  })
</script>
</body></html>




### Image View

html image viewer?


In [ ]:
<head>
  <style>
    ul {
      list-style: none;
      padding: 0;
      margin: 0;
    }

    li {
      padding: 10px;
      border-bottom: 1px solid #ccc;
    }

    li:last-child {
      border-bottom: none;
    }

    i {
      font-size: 18px;
      margin-right: 10px;
      color: #666;
    }

    span {
      font-size: 16px;
      color: #333;
    }

    ul {
      padding-left: 20px;
    }

    ul li {
      padding-left: 10px;
    }
  </style>
  <link rel="stylesheet" type="text/css" href="/amplitude.css" />
  <script src="/amplitude.js"></script>
</head>

<body>
  <h1>Directory Listing</h1>
  <ul>
    <li><a href="./?session={SESSION}">Parent directory/</a></li>
  </ul>
  <img src="{FILE_URL}?raw=true&session={SESSION}" />
</body>

</html>


### rpc index?

show a full rpc index list of services and callout functionality.

ROUTE = /list

ROOT = true


In [ ]:
const path = require('path')
var { listInProject } = importer.import('list project files');
const { functionCache } = importer.import('cache rpc functions with llm descriptions')

let RPC_CACHE

function rpcList(req, res, next) {
  let wrapperFragment = importer.interpret('rpc service index').code
  let repoFragment = importer.interpret('rpc fragment').code
  let repositoriesFragment = importer.interpret('rpc list fragment').code
  let repositoriesHtml = ''

  if (!RPC_CACHE) {
    const notebooks = listInProject(path.resolve(__dirname, '../'), '{,*,*/*,*/*/*,*/*/*/*}*.ipynb')
    for (let i = 0; i < notebooks.length; i++) {
      let cells
      try {
        cells = importer.interpret(path.basename(notebooks[i]))
      } catch (up) {
        if (up.message.includes('Nothing found')) {
          continue
        } else {
          throw up
        }
      }
      for (let j = 0; j < cells.length; j++) {
        const hasFunction = cells[j].code.match(/(export default|export async function|export function|__all__ =|module.exports =)/gi)
        const cache = functionCache[notebooks[i] + '[' + j + ']']
        if (hasFunction && cells[j].questions[0]) {
          repositoriesHtml += repoFragment
            .replaceAll('{REPONAME}', cells[j].questions[0])
            .replaceAll('{USERNAME}', 'rpc')
            .replaceAll('{REPODESC}', cache ? cache.summary.replaceAll('$', '') : 'No description')
        }
      }
    }
  }

  res.send(wrapperFragment
    .replaceAll('{MAINCONTENT}', repositoriesFragment
      .replaceAll('{REPOSITORIES}', repositoriesHtml.replaceAll(/\{[A-Z]*?\}/g, ''))
    ).replaceAll('{USERNAME}', '' /* path.basename(PROFILE_HOME)*/)
    .replaceAll('{TIMESTAMP}', Date.now())
  )
}

module.exports = rpcList



rpc list fragement?


In [ ]:
<section class="repositories">
  <h2>RPC functions</h2>
  <div class="repo-list">
      {REPOSITORIES}
  </div>
</section>



rpc fragement?


In [ ]:
<div class="repo-item">
  <h3><a href="/{USERNAME}/{REPONAME}">{REPONAME}</a></h3>
  <p>{REPODESC}</p>
</div>



rpc service index?


In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Forgotten RPC</title>
    <link rel="stylesheet" href="/express-styles.css?t={TIMESTAMP}">
</head>

<body>
    <!-- Header Section -->
    <header>
        <nav class="navbar">
            <div class="logo">
                <a href="#">Forgotten RPC</a>
            </div>
            <div class="search">
                <input type="text" placeholder="Search GitHub" id="search">
            </div>
            <!--<div class="user-profile">
                <a href="#">User Profile</a>
            </div>-->
        </nav>
    </header>

    <!-- Main Content Section -->
    <div class="container">
        <!-- Sidebar Section -->
        <aside class="sidebar">
            <ul>
                <li><a href="/list">Functions</a></li>
                <li><a href="/swagger">Swagger Spec</a></li>
            </ul>
        </aside>

        <!-- Main Content Area -->
        <main class="main-content">
            {MAINCONTENT}
        </main>
    </div>

    <!-- Footer Section -->
    <footer>
        <p>&copy; 2025 Git Lost. All rights reserved.</p>
    </footer>
    <script>
        document.addEventListener("keydown", function (event) {
            if (event.key === "." && !event.ctrlKey && !event.metaKey) {
                event.preventDefault();
                window.location.href = window.location.href.replace("github.com", "github.dev");
            }
        })
    </script>
</body>

</html>



node express styles?

ROUTE = /express-styles.css


In [ ]:
/* General Reset */
* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

/* Body Styles */
body {
    font-family: Arial, sans-serif;
    background-color: #333;
    color: #FFF;
}

/* Header Styles */
header {
    background-color: #24292f;
    color: #fff;
    padding: 10px 20px;
}

body > header {
    position: fixed;
    top: 0;
    left: 0;
    right: 0;
    z-index: 1000;
}

h1, h2, h3, h4, h5 {
    color: #fff;
    padding: 0 0 20px 0;
}

.navbar {
    display: flex;
    justify-content: space-between;
    align-items: center;
}

.navbar .logo a {
    font-size: 24px;
    color: #fff;
    text-decoration: none;
    font-weight: bold;
}

.navbar .search input {
    padding: 8px 15px;
    font-size: 14px;
    border-radius: 4px;
    border: 1px solid #ddd;
    width: 300px;
}

.navbar .user-profile a {
    color: #fff;
    text-decoration: none;
    font-size: 14px;
}

/* Main Layout */
.container {
    display: flex;
    margin-top: 80px;
    margin-bottom: 80px;
    padding: 0 20px;
}

/* Sidebar Styles */
.sidebar {
    width: 220px;
    background-color: #24292f;
    color: #fff;
    padding-top: 20px;
    margin-right: 20px;
}

.sidebar ul {
    list-style-type: none;
}

.sidebar ul li {
    padding: 10px 10px;
}

.sidebar ul li a {
    color: #fff;
    text-decoration: none;
    font-size: 16px;
    display: block;
    padding: 5px 10px;
}

.repositories h2 {
  font-size: 24px;
  margin-bottom: 20px;
}

.repo-list {
  display: flex;
  flex-direction: column;
  gap: 15px;
}

.repo-item {
  color: #f9fafb;
  padding: 15px;
  border-radius: 6px;
  box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1);
}

.repo-item a {
  color: #f9fafb;
  text-decoration: none;
}

.repo-item a:hover {
  text-decoration: underline;
}

.repo-item:hover {
  background-color: #3c4043;
}
